In [64]:
from txtai.embeddings import Embeddings
from txtai.pipeline import Extractor, LLM
from txtai.pipeline import Textractor

In [65]:
llm = LLM("TheBloke/Mistral-7B-OpenOrca-GGUF/mistral-7b-openorca.Q4_K_M.gguf", verbose=False)

In [66]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tahaelhihi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [67]:
import os

def stream(path):
  for f in sorted(os.listdir(path)):
    fpath = os.path.join(path, f)

    if f.endswith(("context994.txt")):
      print(f"Indexing {fpath}")
      yield textractor(fpath)

textractor = Textractor()


embeddings = Embeddings(content=True)
embeddings.index(stream("."))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Indexing ./clustcontext994.txt
Indexing ./regcontext994.txt


In [68]:
def execute(question, context):
  prompt = f"""<|im_start|>system
{context}

<|im_end|>
<|im_start|>user
# Task:
Analyze the input sentence and extract necessary elements according to the instructions given above. 
Make sure the output strictly follows the specified format without any additional words or deviations.
Input:
{question}

<|im_end|>
<|im_start|>assistant
  """

  return llm(prompt, maxlength=4096)


In [69]:
def context(question):
  results = embeddings.search(question)
  if results:
      context = results[0]["text"]
  else:
      context = "No results found"
  return context

def rag(question):
  return execute(question, context(question))
     

In [70]:

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [71]:
result = rag("Predict height based on volume through regression.") 

In [72]:
print(result)

Regression, dependent attribute: height, independent attributes: volume.


In [73]:
rag("Given the objects dataset, how many types of objects can be found, according to their volume?")

'Clustering attributes: volume'

In [74]:
rag("Predict height based on volume through regression.") 

'Regression, dependent attribute: height, independent attributes: volume'

In [75]:
rag("How many clusters can be identified within the dataset based on volume?")

'Clustering, attributes: volume.'

In [76]:
rag("How many clusters can be identified within the dataset based on volume?")

'Clustering attributes: volume'

In [77]:
rag("How many different types of objects based on volume and height.") 

'Clustering, attributes: volume, height'

In [85]:
rag("Train a model that can estimate volume based on surface given the objects dataset.") 

'Regression, dependent attribute: volume, independent attributes: surface.'

In [79]:
rag("Identify different types of objects based on volume and height.") 

'Clustering, attributes: volume, height'

In [81]:
rag("Cluster objects based on volume and surface area within the dataset.") 

'Clustering attributes: volume, surface'

In [82]:
rag("Estimate height using volume and mass through regression analysis.") 

'Regression, dependent attribute: height, independent attributes: volume, mass.'

In [83]:
rag("Train a model that can estimate mass based on surface given the objects dataset.") 

'Regression:\ndependent attribute: mass,\nindependent attributes: surface.'

In [84]:
rag("Cluster objects based on mass.") 

'Clustering attributes: mass'